# Data Wrangling: Join, Combine 

In many applications, data may be spread across a number of files or databases or be arranged in a form that is not easy to analyze.   
This chapter focuses on tools to help combine, join, and rearrange data. 

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 8.1  Hierarchical Indexing

*Hierarchical indexing* is an important feature of pandas that enables you to have multiple (two or more) index levels on an axis.   
Somewhat abstractly, it provides a way for you to work with higher dimensional data in a lower dimensional form.   
Let’s start with a simple example; create a Series with a list of lists (or arrays) as the index:

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

What you’re seeing is a prettified view of a Series with a MultiIndex as its index.   
The “gaps” in the index display mean “use the label directly above.

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

With a hierarchically indexed object, so-called *partial* indexing is possible, enabling you to concisely select subsets of the data.

In [4]:
data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [5]:
data['b']

1   -0.555730
3    1.965781
dtype: float64

In [6]:
data['b':'c']

b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
dtype: float64

In [7]:
data.loc[['b', 'd']]

b  1   -0.555730
   3    1.965781
d  2    0.281746
   3    0.769023
dtype: float64

In [8]:
data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

Selection is even possible from an “inner” level.

In [9]:
data.loc[:, 2]

a    0.478943
c    0.092908
d    0.281746
dtype: float64

Hierarchical indexing plays an important role in reshaping data and group-based operations like forming a pivot table.   
For example, you could rearrange the data into a DataFrame using its **unstack** method:

In [10]:
data.unstack()

,1,2,3
a,-0.204708,0.478943,-0.519439
b,-0.555730,NaN,1.965781
c,1.393406,0.092908,NaN
d,NaN,0.281746,0.769023


The inverse operation of unstack is stack:


In [11]:
data.unstack().stack()

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

**stack** and **unstack** will be explored in more detail later in this chapter. 

With a DataFrame, either axis can have a hierarchical index:


In [3]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

The hierarchical levels can have names (as strings or any Python objects).   
If so, these will show up in the console output:


In [4]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

With partial column indexing you can similarly select groups of columns:


In [5]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

### Reordering and Sorting Levels

At times you will need to rearrange the order of the levels on an axis or sort the data by the values in one specific level.   
The **swaplevel** takes two level numbers or names and returns a new object with the levels interchanged (but the data is otherwise unaltered):


In [6]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

**sort_index**, on the other hand, sorts the data using only the values in a single level.   
When swapping levels, it’s not uncommon to also use **sort_index** so that the result is lexicographically sorted by the indicated level:


In [7]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [8]:
frame.sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [9]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [10]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [11]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [12]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

<img style="float: left;" src="pic/pic_0_2.png">

Data selection performance is much better on hierarchically indexed objects if the index is lexicographically sorted starting with the outermost level—that is, the result of calling sort_index(level=0) or sort_index().


lexicographical sort: 숫자 < 대문자 < 소문자

In [22]:
'1' < 'a'

True

In [23]:
'a'<'A'

False

### Summary Statistics by Level

Many descriptive and summary statistics on DataFrame and Series have a level option in which you can specify the level you want to aggregate by on a particular axis.   
Consider the above DataFrame; we can aggregate by level on either the rows or columns like so:


In [24]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [25]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [26]:
frame.sum(level='color') #error

KeyError: 'Level color not found'

In [27]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame's columns

It’s not unusual to want to use one or more columns from a DataFrame as the row index; alternatively, you may wish to move the row index into the DataFrame’s columns. 

In [13]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame’s **set_index** function will create a new DataFrame using one or more of its columns as the index

In [14]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

By default the columns are removed from the DataFrame, though you can leave them in:

In [15]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

**reset_index**, on the other hand, does the opposite of set_index; the hierarchical index levels are moved into the columns:


In [16]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


In [17]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

## 8.2  Combining and Merging Datasets

Data contained in pandas objects can be combined together in a number of ways: 
* **pandas.merge** connects rows in DataFrames based on one or more keys. This will be familiar to users of SQL or other relational databases, as it implements database join operations.
* **pandas.concat** concatenates or “stacks” together objects along an axis. 
*  The **combine_first** instance method enables splicing together overlapping data to fill in missing values in one object with values from another.

### Database-Style DataFrame Joins

*Merge* or *join* operations combine datasets by linking rows using one or more *keys*.   
These operations are central to relational databases (e.g., SQL-based).   
The **merge** function in pandas is the main entry point for using these algorithms on your data. 

In [32]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

In [33]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})

In [34]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [35]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


This is an example of a *many-to-one* join; the data in df1 has multiple rows labeled a and b, whereas df2 has only one row for each value in the **key** column.  
Calling merge with these objects we obtain

In [36]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Note that I didn’t specify which column to join on.   
If that information is not specified, **merge** uses the overlapping column names as the keys.   
It’s a good practice to specify explicitly, though.

In [37]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


If the column names are different in each object, you can specify them separately.

In [38]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [39]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [40]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


You may notice that the 'c' and 'd' values and associated data are missing from the result.   
By default **merge** does an 'inner' join; the keys in the result are the intersection, or the common set found in both tables.   
Other possible options are '**left**', '**right**', and '**outer**'. The outer join takes the union of the keys, combining the effect of applying both left and right joins:


In [41]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


See Table 8-1 for a summary of the options for how.


<img style="float: left;" src="pic/pic_8_1.png" width="500">

*Many-to-many* merges have well-defined, though not necessarily intuitive, behavior.

In [42]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})

In [43]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})

In [44]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [45]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [46]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


Many-to-many joins form the Cartesian product of the rows. Since there were three 'b' rows in the left DataFrame and two in the right one, there are six 'b' rows in the result.   
The join method only affects the distinct key values appearing in the result:


In [47]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


To merge with multiple keys, pass a list of column names:

In [48]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

In [49]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [50]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [51]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


To determine which key combinations will appear in the result depending on the choice of merge method, think of the multiple keys as forming an array of tuples to be used as a single join key (even though it’s not actually implemented that way).


A last issue to consider in merge operations is the treatment of overlapping column names.   
While you can address the overlap manually (see the earlier section on renaming axis labels), **merge** has a **suffixes** option for specifying strings to append to overlapping names in the left and right DataFrame objects:


In [52]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [53]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [54]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [55]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


See Table 8-2 for an argument reference on **merge**. 


<img style="float: left;" src="pic/pic_8_2.png" width="700">

### Merging on Index

In some cases, the merge key(s) in a DataFrame will be found in its index.  
In this case, you can pass **left_index=True** or **right_index=True** (or both) to indicate that the index should be used as the merge key:


In [56]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [57]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [58]:
right1

,group_val
a,3.5
b,7.0


In [59]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


Since the default merge method is to intersect the join keys, you can instead form the union of them with an outer join:


In [60]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


With hierarchically indexed data, things are more complicated, as joining on index is implicitly a multiple-key merge:


In [61]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

In [62]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [63]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In this case, you have to indicate multiple columns to merge on as a list (note the handling of duplicate index values with how='outer'):


In [64]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [65]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
         right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


Using the indexes of both sides of the merge is also possible:

In [66]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

In [67]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [68]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [69]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


DataFrame has a convenient **join** instance for merging by index.   
It can also be used to combine together many DataFrame objects having the same or similar indexes but non-overlapping columns.   
In the prior example, we could have written:


In [70]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In part for legacy reasons (i.e., much earlier versions of pandas), DataFrame’s **join** method performs a left join on the join keys, exactly preserving the left frame’s row index.   
It also supports joining the index of the passed DataFrame on one of the columns of the calling DataFrame:


In [71]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [72]:
right1

,group_val
a,3.5
b,7.0


In [73]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


Lastly, for simple index-on-index merges, you can pass a list of DataFrames to join as an alternative to using the more general **concat** function described in the next section:


In [74]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])

In [75]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [76]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [77]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [78]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [79]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


### Concatenating Along an Axis

Another kind of data combination operation is referred to interchangeably as concatenation, binding, or stacking.   
NumPy’s concatenate function can do this with NumPy arrays:


In [80]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [82]:
np.concatenate([arr, arr])

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [81]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In the context of pandas objects such as Series and DataFrame, having labeled axes enable you to further generalize array concatenation. In particular, you have a number of additional things to think about: 

* If the objects are indexed differently on the other axes, should we combine the distinct elements in these axes or use only the shared values (the intersection)? 
* Do the concatenated chunks of data need to be identifiable in the resulting object? 
* Does the “concatenation axis” contain data that needs to be preserved? In many cases, the default integer labels in a DataFrame are best discarded during concatenation.


In [83]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [84]:
s1

a    0
b    1
dtype: int64

In [85]:
s2

c    2
d    3
e    4
dtype: int64

In [86]:
s3

f    5
g    6
dtype: int64

The **concat** function in pandas provides a consistent way to address each of these concerns.   
Suppose we have three Series with no index overlap:


In [87]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

By default **concat** works along axis=0, producing another Series.   
If you pass axis=1, the result will instead be a DataFrame (axis=1 is the columns):


In [88]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In this case there is no overlap on the other axis, which as you can see is the sorted union (the '**outer**' join) of the indexes. You can instead intersect them by passing **join='inner'**:


In [89]:
s1

a    0
b    1
dtype: int64

In [90]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [91]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [92]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In this last example, the 'f' and 'g' labels disappeared because of the **join='inner'** option. 

A potential issue is that the concatenated pieces are not identifiable in the result.   
Suppose instead you wanted to create a hierarchical index on the concatenation axis. To do this, use the keys argument

In [93]:
s1

a    0
b    1
dtype: int64

In [94]:
s3

f    5
g    6
dtype: int64

In [95]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [96]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In the case of combining Series along **axis=1**, the keys become the DataFrame column headers

In [97]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


The same logic extends to DataFrame objects:


In [98]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [99]:
df2

,three,four
a,5,6
c,7,8


In [100]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

If you pass a dict of objects instead of a list, the dict’s keys will be used for the keys option

In [101]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

There are additional arguments governing how the hierarchical index is created (see Table 8-3). 

For example, we can name the created axis levels with the **names** argument:


In [102]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

A last consideration concerns DataFrames in which the row index does not contain any relevant data:


In [103]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df1

,a,b,c,d
0,1.246435,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429,-2.001637
2,-0.371843,1.669025,-0.438570,-0.539741


In [104]:
df2

,b,d,a
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614


In this case, you can pass ignore_index=True:

In [105]:
pd.concat([df1, df2])

,a,b,c,d
0,1.246435,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429,-2.001637
2,-0.371843,1.669025,-0.438570,-0.539741
0,-1.021228,0.476985,NaN,3.248944
1,0.302614,-0.577087,NaN,0.124121


In [106]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,1.246435,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429,-2.001637
2,-0.371843,1.669025,-0.438570,-0.539741
3,-1.021228,0.476985,NaN,3.248944
4,0.302614,-0.577087,NaN,0.124121


<img style="float: left;" src="pic/pic_8_3.png" width="700">

<img style="float: left;" src="pic/pic_8_4.png" width="700">

### Combining Data with Overlap

There is another data combination situation that can’t be expressed as either a merge or concatenation operation.   
You may have two datasets whose indexes overlap in full or part.   
As a motivating example, consider NumPy’s where function, which performs the array-oriented equivalent of an if-else expression:


In [107]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [108]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [109]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

Series has a **combine_first** method, which performs the equivalent of this operation along with pandas’s usual data alignment logic.

Combine two DataFrame objects by filling null values in one DataFrame with non-null values from other DataFrame. The row and column indexes of the resulting DataFrame will be the union of the two.

In [110]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [111]:
b[:-2]

f    0.0
e    1.0
d    2.0
c    3.0
dtype: float64

In [112]:
a[2:]

d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

With DataFrames, **combine_first** does the same thing column by column, so you can think of it as “patching” missing data in the calling object with data from the object you pass

In [113]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [114]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [115]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 8.3   Reshaping and Pivoting

### Reshaping with Hierarchical Indexing

Hierarchical indexing provides a consistent way to rearrange data in a DataFrame. There are two primary actions:

* **stack**  
This “rotates” or pivots from the columns in the data to the rows
* **unstack**   
This pivots from the rows into the columns 

In [116]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


Using the **stack** method on this data pivots the columns into the rows, producing a Series:


In [117]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

From a hierarchically indexed Series, you can rearrange the data back into a DataFrame with **unstack**:


In [118]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [119]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

By default the innermost level is unstacked (same with stack). You can unstack a different level by passing a level number or name:


In [120]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [121]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [122]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


Unstacking might introduce missing data if all of the values in the level aren’t found in each of the subgroups:


In [123]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
s1

a    0
b    1
c    2
d    3
dtype: int64

In [124]:
s2

c    4
d    5
e    6
dtype: int64

In [125]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])

In [126]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [127]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


Stacking filters out missing data by default, so the operation is more easily invertible:

In [128]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [129]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

When you unstack in a DataFrame, the level unstacked becomes the lowest level in the result:


In [130]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [131]:
df.unstack()

side     left           right          
number    one two three   one two three
state                                  
Ohio        0   1     2     5   6     7
Colorado    3   4     5     8   9    10

In [132]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

When calling **stack**, we can indicate the name of the axis to stack

In [133]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

# 이하 생략 #################################

### Pivoting “Long” to “Wide” Format

* A common way to store multiple time series in databases and CSV is in so-called long or stacked format. 

Let’s load some example data and do a small amount of time series wrangling and other data cleaning

In [ ]:
data = pd.read_csv('examples/macrodata.csv')
data.head()

In [ ]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')
periods

In [ ]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
columns

In [ ]:
data = data.reindex(columns=columns)
data

In [ ]:
data.index = periods.to_timestamp('D', 'end')
data.index

In [ ]:
data

In [ ]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})
ldata

We will look at PeriodIndex a bit more closely in Chapter 11. In short, it combines the year and quarter columns to create a kind of time interval type. 

This is the so-called long format for multiple time series, or other observational data with two or more keys (here, our keys are date and item). Each row in the table represents a single observation. 

Data is frequently stored this way in relational databases like MySQL, as a fixed schema (column names and data types) allows the number of distinct values in the **item** column to change as data is added to the table.   
In the previous example, **date** and **item** would usually be the primary keys (in relational database parlance), offering both relational integrity and easier joins.   
In some cases, the data may be more difficult to work with in this format; you might prefer to have a DataFrame containing one column per distinct **item** value indexed by timestamps in the date column.  
DataFrame’s **pivot** method performs exactly this transformation:

In [ ]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

The first two values passed are the columns to be used respectively as the row and column index, then finally an optional value column to fill the DataFrame. 

Suppose you had two value columns that you wanted to reshape simultaneously

In [ ]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

By omitting the last argument, you obtain a DataFrame with hierarchical columns:

In [ ]:
pivoted = ldata.pivot('date', 'item')

In [ ]:
pivoted[:5]

In [ ]:
pivoted['value'][:5]

Note that pivot is equivalent to creating a hierarchical index using **set_index** followed by a call to **unstack**:


In [ ]:
ldata

In [ ]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

### Pivoting “Wide” to “Long” Format

An inverse operation to pivot for DataFrames is **pandas.melt**.   
Rather than transforming one column into many in a new DataFrame, it merges multiple columns into one, producing a DataFrame that is longer than the input. 

In [ ]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

The '**key**' column may be a group indicator, and the other columns are data values. When using **pandas.melt**, we must indicate which columns (if any) are group indicators. Let’s use '**key**' as the only group indicator here:


In [ ]:
melted = pd.melt(df, ['key'])
melted

Using **pivot**, we can reshape back to the original layout:


In [ ]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

Since the result of pivot creates an index from the column used as the row labels, we may want to use **reset_index** to move the data back into a column

In [ ]:
reshaped.reset_index()

You can also specify a subset of columns to use as value columns.

In [ ]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

**pandas.melt** can be used without any group identifiers, too

In [ ]:
pd.melt(df, value_vars=['A', 'B', 'C'])

In [ ]:
df

In [ ]:
pd.melt(df, value_vars=['key', 'A', 'B'])